In [32]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Import CRUD functions
from CRUDing import CRUDopps
# Load the logo for project
image_filename = 'logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "password"
shelter = CRUDopps(username, password)


# Read da from MongoDB and create a DF
df = pd.DataFrame.from_records(shelter.read({}))
df.drop(columns=['_id'], inplace=True)
df['selected']=False #this gives a selected column

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div([
        # Include Grazioso Salvare logo
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'display': 'block', 'margin-left': 'auto', 'margin-right': 'auto', 'width': '5%', 'height': '5%'}),
        # Centered title
        html.Center(html.B(html.H1('Animal Rescue Dashboard'))),
    ]),
    html.Hr(),
    html.Div([
        # Added interactive filtering options (radio items)
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water_rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain_rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster_tracking'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',  # Default value
            labelStyle={'display': 'inline-block', 'margin-right': '10px'}
        )
    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        selected_rows=[0],  # Set the default selected row to the first row
        style_table={'overflowX': 'auto'},  # Add this line for horizontal scroll
        style_cell={'textAlign': 'left'},  # Align cell text to the left
        # Add a new column with checkboxes for row selection
        style_data_conditional=[
            {
                'if': {'column_id': 'selected'},
                'backgroundColor': '#FF8552',
                'color': 'white',
            }
        ],
        editable=True,
        row_selectable='single',
    ),
    html.Br(),
    html.Hr(),
    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            children=[
                dcc.Graph(
                    figure=px.pie(df, names='breed', title='Preferred Animals')
                )
            ]
        ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            children=[
                dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
                    dl.TileLayer(id="base-layer-id"),
                    dl.Marker(position=[30.75, -97.48], children=[
                        dl.Tooltip("Austin, TX"),
                        dl.Popup([
                            html.H1("Location"),
                            html.P("Austin, TX")
                        ])
                    ])
                ])
            ]
        )
    ])
])

# Add a filtering function:
def filter_data(filter_type, data):
    if filter_type == 'reset':
        return data  # No filtering, return the original data

    df = pd.DataFrame.from_records(data)  # Convert list of dictionaries to DataFrame

    if filter_type.lower() == 'water_rescue':
        # Filter for specific breeds in Water Rescue category
        breeds_to_include = ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']
        filtered_data = df[(df['breed'].isin(breeds_to_include)) & (df['animal_type'] == 'Dog')]
    elif filter_type.lower() == 'mountain_rescue':
        # Filter for specific breeds in Mountain or Wilderness Rescue category
        breeds_to_include = ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']
        filtered_data = df[(df['breed'].isin(breeds_to_include)) & (df['animal_type'] == 'Dog')]
    elif filter_type.lower() == 'disaster_tracking':
        # Filter for specific breeds in Disaster or Individual Tracking category
        breeds_to_include = ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']
        filtered_data = df[(df['breed'].isin(breeds_to_include)) & (df['animal_type'] == 'Dog')]
    else:
        filtered_data = df  # Default case, no filtering

    return filtered_data.to_dict('records')  # Convert DataFrame back to list of dictionaries




#############################################
# Interaction Between Components / Controller
#############################################

# Update the data table based on the selected filter type    
@app.callback(Output('datatable-id', 'data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    filtered_data = filter_data(filter_type, df.to_dict('records'))
    return filtered_data

# Update the charts based on the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None:
        return

    dff = pd.DataFrame.from_dict(viewData)

    # Calculate the top 10 highest percentages
    top_10_breeds = dff['breed'].value_counts(normalize=True).nlargest(10).index

    # Filter the data for the top 10 breeds
    filtered_data = dff[dff['breed'].isin(top_10_breeds)]

    return [
        dcc.Graph(
            figure=px.pie(filtered_data, names='breed', title='Top 10 Preferred Animals by Benjamin Leanna')
        )
    ]


#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []

    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]



# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "selected_rows")])
def update_map(viewData, selected_rows):  
    if viewData is None or not selected_rows or selected_rows[0] >= len(viewData):
        return []

    dff = pd.DataFrame.from_dict(viewData)
    row = selected_rows[0]
    
    # Extract location information from the selected row
    location_lat = dff.iloc[row, 13]
    location_long = dff.iloc[row, 14]

    # Update the map with the selected location
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[location_lat, location_long], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[location_lat, location_long], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]

app.run_server(debug=True)


Dash app running on http://127.0.0.1:8305/
